### Import Required Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "../../../"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [6]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

### Train Model on Data

In [7]:
er_df = ERData_MajorityLabel()

In [8]:
train, test = train_test_split(er_df, test_size=0.3)

In [9]:
print(train.shape, ":", test.shape)

(5004642, 2) : (2144847, 2)


In [10]:
# train, val, test = splitTrainTestVal(er_df[:500000], 0.8 ,0, 'Name')

In [11]:
# train1, _, test1 = splitTrainTestVal(er_df[100000:500000], 0.8 ,0, 'Name')

In [12]:
from sklearn.svm import SVC

In [13]:
svm_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6), lowercase = True, stop_words = "english")),
    ('tfidf', TfidfTransformer(sublinear_tf = True)),
    ('clf', SVC(max_iter=10000, random_state=42))
])

In [14]:
svm_pipe.fit(train['Name'].values.astype('str'), train['Gender'])

/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 6), stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', SVC(max_iter=10000, random_state=42))])

In [15]:
y_pred=svm_pipe.predict(test['Name'].values.astype('str'))
print(classification_report(test['Gender'], y_pred, target_names = ['Female', 'Male'], digits=4))

              precision    recall  f1-score   support

      Female     0.5690    0.7389    0.6429   1000036
        Male     0.6915    0.5111    0.5878   1144811

    accuracy                         0.6173   2144847
   macro avg     0.6302    0.6250    0.6153   2144847
weighted avg     0.6344    0.6173    0.6135   2144847



### Save Model

In [17]:
filename = "Models/ERData/SVM/SavedModel/ERModel.pkl"
pickle.dump(svm_pipe, open(filename, 'wb'))